In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

from catboost import CatBoostClassifier
from sklearn.utils.class_weight import compute_class_weight

train_path = './data/train.csv'
test_path = './data/test.csv'
sample_path = './data/sample_submission.csv'

## LGBM

In [17]:
from lgbm_process import lgbm_process

train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])
train, test = lgbm_process(train, test)
print(train.shape, test.shape)

(256351, 67) (90067, 66)


In [ ]:
seed_list = [333]
folds = [10]

In [28]:
train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

total_auc, total_acc, total_f1 = [], [], []
test_preds_lgbm = []

for seed in seed_list:
    for k in folds:
        skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
        auc_scores, acc_scores,  f1_scores = [], [], []

        for fold, (train_idx, valid_idx) in enumerate(skf.split(train.drop(columns=['임신 성공 여부']), train["임신 성공 여부"])):
            fold_train = train.iloc[train_idx].copy().reset_index(drop=True)
            fold_train2 = train.iloc[train_idx].copy().reset_index(drop=True)
            fold_valid = train.iloc[valid_idx].copy().reset_index(drop=True)
            fold_test = test.copy()

            fold_train, fold_valid = lgbm_process(fold_train, fold_valid, seed=seed)
            fold_train2, fold_test = lgbm_process(fold_train2, fold_test, seed=seed)

            X_train = fold_train.drop(columns=['임신 성공 여부'])
            X_valid = fold_valid.drop(columns=['임신 성공 여부'])
            y_train = fold_train['임신 성공 여부']
            y_valid = fold_valid['임신 성공 여부']

            lgbm_params_42 = {
                'n_estimators': 1134,
                'learning_rate': 0.009183378614268902,
                'max_depth': 15,
                'num_leaves': 59,
                'min_child_samples': 56,
                'subsample': 0.5894604069264655,
                'colsample_bytree': 0.6305670256882752,
                'reg_alpha': 7.47936987466662,
                'reg_lambda': 0.0010986427203281623,
            }

            model_lgb = LGBMClassifier(
                **lgbm_params_42,
                verbosity=-1,
                n_jobs=-1,
                random_state=seed,
            )

            model_lgb.fit(
                X_train, y_train,
            )

            valid_preds_proba  = model_lgb.predict_proba(X_valid)[:, 1]
            valid_preds_class = model_lgb.predict(X_valid)  # 클래스 예측값 (0 또는 1)

            # AUC, Accuracy, F1-score 계산
            auc_ = roc_auc_score(y_valid, valid_preds_proba)
            acc_ = accuracy_score(y_valid, valid_preds_class)
            f1_ = f1_score(y_valid, valid_preds_class)

            print(f"Seed[{seed:<3}] Fold {fold + 1} | AUC: {auc_:.7f} | Acc: {acc_:.7f} | F1: {f1_:.7f}")

            auc_scores.append(auc_)
            acc_scores.append(acc_)
            f1_scores.append(f1_)

            total_auc.append(auc_)
            total_acc.append(acc_)
            total_f1.append(f1_)

            test_pred = model_lgb.predict_proba(fold_test)[:, 1]
            test_preds_lgbm.append(test_pred)

        # fold 별 평균 성능 계산
        avg_auc = np.mean(auc_scores)
        avg_acc = np.mean(acc_scores)
        avg_f1 = np.mean(f1_scores)

        print("-" * 80)
        print(f"Seed[{seed:<3}] Average Metrics | AUC: {avg_auc:.7f} | Acc: {avg_acc:.7f} | F1: {avg_f1:.7f}")
        print("-" * 80)

val_auc = np.mean(total_auc)
val_acc = np.mean(total_acc)
val_f1 = np.mean(total_f1)

print("-" * 80)
print(f"Validation Average Metrics | AUC: {val_auc:.7f} | Acc: {val_acc:.7f} | F1: {val_f1:.7f}")

Seed[4  ] Fold 1 | AUC: 0.7374502 | Acc: 0.7468404 | F1: 0.1999507
Seed[4  ] Fold 2 | AUC: 0.7447556 | Acc: 0.7456212 | F1: 0.1956334
Seed[4  ] Fold 3 | AUC: 0.7414125 | Acc: 0.7440999 | F1: 0.1907229
Seed[4  ] Fold 4 | AUC: 0.7450743 | Acc: 0.7469475 | F1: 0.1952611
Seed[4  ] Fold 5 | AUC: 0.7402540 | Acc: 0.7456212 | F1: 0.1861974
Seed[4  ] Fold 6 | AUC: 0.7354118 | Acc: 0.7454262 | F1: 0.1913259
Seed[4  ] Fold 7 | AUC: 0.7357952 | Acc: 0.7474547 | F1: 0.2052541
Seed[4  ] Fold 8 | AUC: 0.7327653 | Acc: 0.7462064 | F1: 0.2011297
Seed[4  ] Fold 9 | AUC: 0.7440123 | Acc: 0.7464794 | F1: 0.1981493
Seed[4  ] Fold 10 | AUC: 0.7445284 | Acc: 0.7482348 | F1: 0.2073201
--------------------------------------------------------------------------------
Seed[4  ] Average Metrics | AUC: 0.7401460 | Acc: 0.7462932 | F1: 0.1970945
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Validation

In [25]:
tmp_submission = pd.DataFrame({f'lgbm_{seed_list[0]}': np.mean(test_preds_lgbm, axis=0)})
tmp_submission

,lgbm_3
0,0.001666
1,0.002246
2,0.155921
3,0.097274
4,0.529565
...,...
90062,0.001592
90063,0.294332
90064,0.470224
90065,0.182592


## Catboost

In [26]:
from CB_Process import cb_all_process
train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])

train, test = cb_all_process(train, test)

print(train.shape, test.shape)

(256351, 65) (90067, 64)


In [27]:
train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

total_auc, total_acc, total_f1 = [], [], []
test_preds_cat = []

is_first = True
for seed in seed_list:
    for k in folds:
        skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
        auc_scores, acc_scores,  f1_scores = [], [], []

        for fold, (train_idx, valid_idx) in enumerate(skf.split(train.drop(columns=['임신 성공 여부']), train["임신 성공 여부"])):
            fold_train = train.iloc[train_idx].copy().reset_index(drop=True)
            fold_train2 = train.iloc[train_idx].copy().reset_index(drop=True)
            fold_valid = train.iloc[valid_idx].copy().reset_index(drop=True)
            fold_test = test.copy()

            fold_train, fold_valid = cb_all_process(fold_train, fold_valid)
            fold_train2, fold_test = cb_all_process(fold_train2, fold_test)

            X_train = fold_train.drop(columns=['임신 성공 여부'])
            X_valid = fold_valid.drop(columns=['임신 성공 여부'])
            y_train = fold_train['임신 성공 여부']
            y_valid = fold_valid['임신 성공 여부']

            cat_features = X_train.select_dtypes(include=["object", "category"]).columns.tolist()

            if is_first:
                cat_features = X_train.select_dtypes(include=["object", "category"]).columns.tolist()
                print(f'범주형 변수: {len(cat_features)}개 \n {cat_features} \n')

                num_features = [col for col in X_train.columns if col not in cat_features + ['임신 성공 여부']]
                print(f'수치형 변수: {len(num_features)}개 \n {num_features} \n')
                is_first = False

            weights_sqrt = np.sqrt(compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train))

            # Catboost 모델 초기화
            model_cat = CatBoostClassifier(iterations=2000, learning_rate=0.05, random_seed=seed,
                                       loss_function='Logloss', eval_metric='Logloss', class_weights=weights_sqrt,
                                       cat_features=cat_features, thread_count=-1)

            model_cat.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=1000)

            valid_preds_proba = model_cat.predict_proba(X_valid)[:, 1]
            valid_preds_class = model_cat.predict(X_valid)  # 클래스 예측값 (0 또는 1)

            # AUC, Accuracy, F1-score 계산
            auc_ = roc_auc_score(y_valid, valid_preds_proba)
            acc_ = accuracy_score(y_valid, valid_preds_class)
            f1_ = f1_score(y_valid, valid_preds_class)

            print(f"Seed[{seed:<3}] Fold {fold + 1} | AUC: {auc_:.7f} | Acc: {acc_:.7f} | F1: {f1_:.7f}")

            auc_scores.append(auc_)
            acc_scores.append(acc_)
            f1_scores.append(f1_)

            total_auc.append(auc_)
            total_acc.append(acc_)
            total_f1.append(f1_)

            test_pred = model_cat.predict_proba(fold_test)[:, 1]
            test_preds_cat.append(test_pred)

        # fold 별 평균 성능 계산
        avg_auc = np.mean(auc_scores)
        avg_acc = np.mean(acc_scores)
        avg_f1 = np.mean(f1_scores)

        print("-" * 80)
        print(f"Seed[{seed:<3}] Average Metrics | AUC: {avg_auc:.7f} | Acc: {avg_acc:.7f} | F1: {avg_f1:.7f}")
        print("-" * 80)

val_auc = np.mean(total_auc)
val_acc = np.mean(total_acc)
val_f1 = np.mean(total_f1)

print("-" * 80)
print(f"Validation Average Metrics | AUC: {val_auc:.7f} | Acc: {val_acc:.7f} | F1: {val_f1:.7f}")

범주형 변수: 18개 
 ['시술 시기 코드', '시술 당시 나이', '배란 유도 유형', '배아 생성 주요 이유', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이', '시술유형_통합'] 

수치형 변수: 46개 
 ['임신 시도 또는 마지막 임신 경과 연수', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 채취 경과일', '난자 해동 경과일', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일', '시술_임신'] 

0:	learn: 0.6737465	test: 0.6738002	best: 0.6738002 (

KeyboardInterrupt: 

In [1]:
tmp_submission[f'cat_{seed_list[0]}'] = np.mean(test_preds_cat, axis=0)

NameError: name 'np' is not defined

## Ensemble

In [10]:
tmp_submission

,lgbm_666,cat_666
0,0.001867,0.002123
1,0.002010,0.014248
2,0.158129,0.231421
3,0.096000,0.166438
4,0.520879,0.630755
...,...,...
90062,0.001539,0.002472
90063,0.293352,0.454370
90064,0.467433,0.548639
90065,0.174970,0.323370


In [11]:
tmp_submission['row_avg'] = tmp_submission.mean(axis=1)

In [12]:
tmp_submission

,lgbm_666,cat_666,row_avg
0,0.001867,0.002123,0.001995
1,0.002010,0.014248,0.008129
2,0.158129,0.231421,0.194775
3,0.096000,0.166438,0.131219
4,0.520879,0.630755,0.575817
...,...,...,...
90062,0.001539,0.002472,0.002005
90063,0.293352,0.454370,0.373861
90064,0.467433,0.548639,0.508036
90065,0.174970,0.323370,0.249170


In [13]:
seed=808
tmp_submission.to_csv(f'seed_{seed}_probability.csv', index=False, encoding='utf-8-sig')

In [14]:
final_pred = tmp_submission.loc[:, 'row_avg']

In [15]:
submit = pd.read_csv(sample_path)
submit['probability'] = final_pred
submit

,ID,probability
0,TEST_00000,0.001995
1,TEST_00001,0.008129
2,TEST_00002,0.194775
3,TEST_00003,0.131219
4,TEST_00004,0.575817
...,...,...
90062,TEST_90062,0.002005
90063,TEST_90063,0.373861
90064,TEST_90064,0.508036
90065,TEST_90065,0.249170


In [16]:
submit.to_csv('./666666666666666.csv',index=False)